# MovieLens 100K Recommender System - Phase 1
## Data Loading, EDA, and Baseline Models

**Project:** Hybrid Movie Recommendation System  
**Dataset:** MovieLens 100K  
**Author:** Harpinder Singh  
**Date:** December 2025  

## ╔═══════════════════════════════════════════════════════════╗
## ║                    IMPORTS & SETUP                        ║
## ╚═══════════════════════════════════════════════════════════╝

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
import zipfile
import os
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')
np.random.seed(42)

# Plot styling
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Libraries imported successfully")
print(f"📅 Execution Date: {datetime.now().strftime('%B %d, %Y')}")

✅ Libraries imported successfully
📅 Execution Date: February 01, 2026


## ╔═══════════════════════════════════════════════════════════╗
## ║              DATA DOWNLOAD & EXTRACTION                   ║
## ╚═══════════════════════════════════════════════════════════╝

In [2]:
def download_movielens_100k():
    """
    Download and extract MovieLens 100K dataset.
    
    Returns
    -------
    str
        Path to extracted data directory
    """
    url = 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'
    zip_path = '../data/ml-100k.zip'
    extract_path = '../data/ml-100k'
    
    # Download if not exists
    if not os.path.exists(zip_path):
        print("📥 Downloading MovieLens 100K dataset...")
        urllib.request.urlretrieve(url, zip_path)
        print("✅ Download complete")
    else:
        print("✅ Dataset already downloaded")
    
    # Extract if not exists
    if not os.path.exists(extract_path):
        print("📂 Extracting files...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall('../data')
        print("✅ Extraction complete")
    else:
        print("✅ Files already extracted")
    
    return extract_path

data_path = download_movielens_100k()
print(f"\n📁 Data directory: {data_path}")

✅ Dataset already downloaded
📂 Extracting files...
✅ Extraction complete

📁 Data directory: ../data/ml-100k


## ╔═══════════════════════════════════════════════════════════╗
## ║                  LOAD INDIVIDUAL FILES                    ║
## ╚═══════════════════════════════════════════════════════════╝

In [3]:
# ┌─────────────────────────────────────────┐
# │ Load Ratings (u.data)                   │
# └─────────────────────────────────────────┘

ratings = pd.read_csv(
    f'{data_path}/u.data',
    sep='\t',
    names=['user_id', 'item_id', 'rating', 'timestamp'],
    encoding='latin-1'
)

print("📊 Ratings Data:")
print(f"   Shape: {ratings.shape}")
print(f"   Columns: {list(ratings.columns)}")
print("\nFirst 5 rows:")
print(ratings.head())

📊 Ratings Data:
   Shape: (100000, 4)
   Columns: ['user_id', 'item_id', 'rating', 'timestamp']

First 5 rows:
   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596


In [4]:
# ┌─────────────────────────────────────────┐
# │ Load Movies (u.item)                    │
# └─────────────────────────────────────────┘

# Genre columns (19 genres)
genre_cols = [
    'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy',
    'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
    'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
]

movies = pd.read_csv(
    f'{data_path}/u.item',
    sep='|',
    names=['item_id', 'title', 'release_date', 'video_release_date', 'imdb_url'] + genre_cols,
    encoding='latin-1'
)

print("\n🎬 Movies Data:")
print(f"   Shape: {movies.shape}")
print(f"   Columns: {list(movies.columns[:10])}...")  # Show first 10 columns
print("\nFirst 5 rows (selected columns):")
print(movies[['item_id', 'title', 'Action', 'Comedy', 'Drama']].head())


🎬 Movies Data:
   Shape: (1682, 24)
   Columns: ['item_id', 'title', 'release_date', 'video_release_date', 'imdb_url', 'unknown', 'Action', 'Adventure', 'Animation', 'Children']...

First 5 rows (selected columns):
   item_id              title  Action  Comedy  Drama
0        1   Toy Story (1995)       0       1      0
1        2   GoldenEye (1995)       1       0      0
2        3  Four Rooms (1995)       0       0      0
3        4  Get Shorty (1995)       1       1      1
4        5     Copycat (1995)       0       0      1


In [5]:
# ┌─────────────────────────────────────────┐
# │ Load Users (u.user)                     │
# └─────────────────────────────────────────┘

users = pd.read_csv(
    f'{data_path}/u.user',
    sep='|',
    names=['user_id', 'age', 'gender', 'occupation', 'zip_code'],
    encoding='latin-1'
)

print("\n👥 Users Data:")
print(f"   Shape: {users.shape}")
print(f"   Columns: {list(users.columns)}")
print("\nFirst 5 rows:")
print(users.head())


👥 Users Data:
   Shape: (943, 5)
   Columns: ['user_id', 'age', 'gender', 'occupation', 'zip_code']

First 5 rows:
   user_id  age gender  occupation zip_code
0        1   24      M  technician    85711
1        2   53      F       other    94043
2        3   23      M      writer    32067
3        4   24      M  technician    43537
4        5   33      F       other    15213


## ╔═══════════════════════════════════════════════════════════╗
## ║              CREATE MERGED DATASET                        ║
## ╚═══════════════════════════════════════════════════════════╝

In [6]:
# ┌─────────────────────────────────────────┐
# │ Create Genre String Column              │
# └─────────────────────────────────────────┘

def get_genres(row):
    """
    Extract genre string from binary genre columns.
    
    Parameters
    ----------
    row : pd.Series
        Row containing binary genre indicators
    
    Returns
    -------
    str
        Pipe-separated genre string
    """
    genres = [genre for genre in genre_cols if row[genre] == 1]
    return '|'.join(genres) if genres else 'unknown'

movies['genres'] = movies.apply(get_genres, axis=1)

print("🎭 Sample movie genres:")
print(movies[['title', 'genres']].head(10))

🎭 Sample movie genres:
                                               title  \
0                                   Toy Story (1995)   
1                                   GoldenEye (1995)   
2                                  Four Rooms (1995)   
3                                  Get Shorty (1995)   
4                                     Copycat (1995)   
5  Shanghai Triad (Yao a yao yao dao waipo qiao) ...   
6                              Twelve Monkeys (1995)   
7                                        Babe (1995)   
8                            Dead Man Walking (1995)   
9                                 Richard III (1995)   

                      genres  
0  Animation|Children|Comedy  
1  Action|Adventure|Thriller  
2                   Thriller  
3        Action|Comedy|Drama  
4       Crime|Drama|Thriller  
5                      Drama  
6               Drama|Sci-Fi  
7      Children|Comedy|Drama  
8                      Drama  
9                  Drama|War  


In [7]:
# ┌─────────────────────────────────────────┐
# │ Merge All Data                          │
# └─────────────────────────────────────────┘

# Select relevant movie columns
movies_clean = movies[['item_id', 'title', 'genres']]

# Merge ratings with movies
data = ratings.merge(movies_clean, on='item_id', how='left')

# Merge with users
data = data.merge(users, on='user_id', how='left')

print("🔗 Merged Dataset:")
print(f"   Shape: {data.shape}")
print(f"   Columns: {list(data.columns)}")
print("\nFirst 5 rows:")
print(data.head())

print("\n📊 Data Types:")
print(data.dtypes)

🔗 Merged Dataset:
   Shape: (100000, 10)
   Columns: ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres', 'age', 'gender', 'occupation', 'zip_code']

First 5 rows:
   user_id  item_id  rating  timestamp                       title  \
0      196      242       3  881250949                Kolya (1996)   
1      186      302       3  891717742    L.A. Confidential (1997)   
2       22      377       1  878887116         Heavyweights (1994)   
3      244       51       2  880606923  Legends of the Fall (1994)   
4      166      346       1  886397596         Jackie Brown (1997)   

                             genres  age gender  occupation zip_code  
0                            Comedy   49      M      writer    55105  
1  Crime|Film-Noir|Mystery|Thriller   39      F   executive    00000  
2                   Children|Comedy   25      M      writer    40206  
3         Drama|Romance|War|Western   28      M  technician    80525  
4                       Crime|Drama   47      M 

In [8]:
# ┌─────────────────────────────────────────┐
# │ Save Merged CSV                         │
# └─────────────────────────────────────────┘

output_path = '../data/movielens_100k_merged.csv'

data.to_csv(output_path, index=False)
print("💾 Saved merged dataset: movielens_100k_merged.csv")
print(f"   File size: {os.path.getsize(output_path) / (1024*1024):.2f} MB")

💾 Saved merged dataset: movielens_100k_merged.csv
   File size: 7.70 MB


## ╔═══════════════════════════════════════════════════════════╗
## ║           EXPLORATORY DATA ANALYSIS (EDA)                 ║
## ╚═══════════════════════════════════════════════════════════╝

In [9]:
# ┌─────────────────────────────────────────┐
# │ Basic Statistics                        │
# └─────────────────────────────────────────┘

print("📊 DATASET STATISTICS")
print("=" * 50)
print(f"Total Ratings:        {len(data):,}")
print(f"Unique Users:         {data['user_id'].nunique():,}")
print(f"Unique Movies:        {data['item_id'].nunique():,}")
print(f"Rating Range:         {data['rating'].min()} - {data['rating'].max()}")
print(f"Average Rating:       {data['rating'].mean():.2f}")
print(f"Median Rating:        {data['rating'].median():.1f}")
print(f"\nDate Range:")
print(f"  Earliest:           {pd.to_datetime(data['timestamp'], unit='s').min()}")
print(f"  Latest:             {pd.to_datetime(data['timestamp'], unit='s').max()}")

# Calculate sparsity
n_users = data['user_id'].nunique()
n_items = data['item_id'].nunique()
n_ratings = len(data)
sparsity = 100 * (1 - n_ratings / (n_users * n_items))

print(f"\nMatrix Sparsity:      {sparsity:.2f}%")
print(f"Matrix Density:       {100 - sparsity:.2f}%")

📊 DATASET STATISTICS
Total Ratings:        100,000
Unique Users:         943
Unique Movies:        1,682
Rating Range:         1 - 5
Average Rating:       3.53
Median Rating:        4.0

Date Range:
  Earliest:           1997-09-20 03:05:10
  Latest:             1998-04-22 23:10:38

Matrix Sparsity:      93.70%
Matrix Density:       6.30%
